In [4]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import csv
import re


def pop():
    # 맨 위 뉴스 팝업
    time.sleep(2)
    first = browser.find_element(By.CSS_SELECTOR, '#news-results > div:nth-child(1) > div > div.cont > a > div > strong > span')
    time.sleep(2)
    first.click()

def gettitle():
    news_title = browser.find_element(By.CSS_SELECTOR, '#news-detail-modal > div > div > div.modal-body > div > div.news-view-head > h1.title')
    print('newsTitle:', news_title.text)
    tmp["newsTitle"].append(news_title.text)

def getbody():
    # 날짜 저장
    news_date = browser.find_elements(By.XPATH, '//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/div[1]/ul/li[1]')
    for value in news_date:
        print('newsDate:',value.text)
        tmp["newsDate"].append(value.text)
    # 하나의 뉴스기사 기준 본문 저장
    news_bodies = browser.find_elements(By.XPATH, '//*[@id="news-detail-modal"]/div/div/div[1]/div/div[2]')
    news_body =''
    for i in news_bodies:
        news_body += i.text
    print('newsBody:',news_body[:50]," ...중략...")
    tmp["newsBody"].append(news_body)


# selenium
browser = webdriver.Chrome()
# wait=WebDriverWait(browser, 10)

browser.set_window_size(1200,800)
url = "https://www.bigkinds.or.kr/v2/news/search.do"

browser.get(url)
time.sleep(2)

# 기간 설정
fromdate = "\'2018-01-01\'"
todate ="\'2018-01-01\'"

day31 = [1,3,5,7,8,10,12]
day30 = [2,4,6,9,11]


browser.find_element(By.XPATH, '//*[@id="collapse-step-1"]').click()
browser.find_element(By.XPATH, '//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
time.sleep(2)

from_ = browser.find_element(By.XPATH, '//*[@id="search-begin-date"]')
browser.execute_script("arguments[0].value = {};".format(fromdate), from_)
to_ = browser.find_element(By.XPATH, '//*[@id="search-end-date"]')
browser.execute_script("arguments[0].value = {};".format(todate), to_)

# 사건사고 분류 카테고리 - 범죄 선택
browser.find_element(By.XPATH, '//*[@id="collapse-step-1-body"]/div[3]/div/div[2]/div[3]/a').click()
browser.find_element(By.XPATH,'//*[@id="srch-tab4"]/ul/li[1]/div/span[4]').click()
time.sleep(2)
browser.find_element(By.XPATH,'//*[@id="search-foot-div"]/div[2]/button[2]').click()
time.sleep(2)

# 빈 데이터셋 생성
tmp = {"newsId":[],"newsTitle":[],"newsDate":[],"newsBody":[]}


newsId = 0
# iteration = 0

while (True):
    try:
        print('newsId:',newsId)
        tmp["newsId"].append(newsId)
        time.sleep(2)
        if (newsId % 10 == 0):
            pop()
        time.sleep(1)
        gettitle()
        getbody()
        # 다음 뉴스 글로 넘기기
        if (newsId == 0):
            browser.find_element(By.CSS_SELECTOR,
                '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li:nth-child(2) > dd > a > span').click()
        elif (newsId % 10 == 9 or newsId % 10 == 0 ):
            next_news = browser.find_element(By.CSS_SELECTOR, 
                '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li:nth-child(3) > dd > a > span')
            time.sleep(1)
            next_news.click()
        else:
            next_news = browser.find_element(By.CSS_SELECTOR,
                '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li.nextNewsItem > dd > a > span')
            time.sleep(1)
            next_news.click()
        newsId += 1
        # save to csv
        df = pd.DataFrame(tmp)
        if not os.path.exists("dataset.csv"):
            df.to_csv("dataset.csv", index=False, mode="w", encoding="utf-8-sig")
        else:
            df.to_csv('dataset.csv', index=False, mode="a", encoding="utf-8-sig", header=False)
    except:
        newsId += 1
    finally:
        if (newsId > 2 and tmp["newsTitle"][-1] == tmp["newsTitle"][-2] == tmp["newsTitle"][-3]):
            break
    

print("-"*40,"Complete to acquisite.","-"*40)

newsId: 0
newsTitle: 일본 야권 거물 정치인 “아베, 박근혜보다도 권력 사유화”
newsDate: 2018-01-01
newsBody: 일본 야권의 중견 의원이 1일 아베 신조(安倍晋三) 일본 총리에 대해 박근혜(66) 전 대  ...중략...
newsId: 1
newsTitle: 조정석X김선호X이혜리, 믿음과 진실 향해 한발짝 더(종합)
newsDate: 2018-01-01
newsBody: ‘투깝스’ 조정석이 진실에 조금 더 가까워졌다.

1일 밤 10시 방송한 MBC 월화드라마  ...중략...
newsId: 2
newsTitle: 충주 노부부 피살 “땅 매각 갈등 막내아들이 살해”
newsDate: 2018-01-01
newsBody: 경찰 구속영장 신청, 막내아들은 범행 부인

충북 충주경찰서는 지난달 27일 충주에서 발생  ...중략...
newsId: 3
newsTitle: 뷰티샵에서 6천만원 빼돌린 피부관리사 징역형
newsDate: 2018-01-01
newsBody: 40대 피부관리사가 피부 미용을 관리하는 '뷰티샵'에서 점장을 맡아 일하다가 수천만원을 빼  ...중략...
newsId: 4
newsTitle: “화재 신고 후 혼자 빠져 나왔다” 3남매 친모 진술 번복
newsDate: 2018-01-01
newsBody: 경찰, 구속영장 신청
“정씨 진술 믿기 어려워” 방화 여부 수사 집중

광주에서 발생한 화  ...중략...
newsId: 5
newsTitle: [법에 비친 세상] 하차 요구 거부한 택시기사, 감금죄 무죄인 까닭은
newsDate: 2018-01-01
newsBody: “내려달라” 무시하고 목적지까지
법원 “경로 이탈 없이 안전 운행”
남편과 통화하며 구조요  ...중략...
newsId: 6
newsTitle: 문 대통령, 북한산 해돋이 보며 새해 맞이
newsDate: 2018-01-01
newsBody: 2017년을 빛낸 의인들 동행... 관저서 떡국 대접
사모바위서 손학규 국민의당 상임고문과  ..

newsId: 57
newsTitle: 검찰, 살인죄 구형 대폭 높인다… 아동 납치살인 등 최대 사형
newsDate: 2018-01-01
newsBody: 검찰은 1일 연이은 강력범죄에 대응하기 위해 최대 사형 구형을 하겠다는 방침을 세웠다.

  ...중략...
newsId: 58
newsTitle: ‘투깝스’ 진실을 밝히려는 조정석, 묻으려는 정해균과 싸늘한 냉기
newsDate: 2018-01-01
newsBody: ‘투깝스’ 조정석과 정해균, 그들 사이에 다시 싸늘한 냉기가 흐르
고 있다.

새해에도 조  ...중략...
newsId: 59
newsTitle: 문 대통령 "국민 삶의 질 개선"…의인들과 해맞이
newsDate: 2018-01-01
newsBody: 【앵커】

문재인 대통령은 신년사를 통해 올해 최우선 국정목표를 '국민의 삶의 질 개선'에  ...중략...
newsId: 60
newsTitle: 檢, 아동 납치살인 등 최대 사형까지 구형
newsDate: 2018-01-01
newsBody: 검찰이 미성년자 납치 살해나 강간 살해 등 인명 경시 성향이 강한 범죄에 대해서는 무기징역  ...중략...
newsId: 61
newsTitle: [2018 경향 신춘문예]평론부문 당선작 | 인아영 ‘유토피아에서 있었던 일에 대해 알아보려고 합니다(박민정론)’
newsDate: 2018-01-01
newsBody: 일러스트 | 김상민 기자
자, 이제 내 몸은 이 모든 유토피아들 덕분에 사라졌다.

-미셸  ...중략...
newsId: 62
newsTitle: 충주 노부부 피살사건 용의자 막내아들 구속영장 신청
newsDate: 2018-01-01
newsBody: 지난달 충북 충주에서 발생한 노부부 피살 사건의 유력한 용의자인 막내아들 김모씨(46)에   ...중략...
newsId: 63
newsTitle: 충주 80대 노부부 피살사건 용의자 막내아들 구속영장
newsDate: 2018-01-01
newsBody: 지난달 27일 충북 충주에서 

newsId: 114
newsTitle: 아동 납치, 성폭행 동반 살인하면 최소 무기징역 구형한다
newsDate: 2018-01-01
newsBody: 검찰, 강화된 살인 구형 기준 1일부터 전국 시행 강력범죄 동반 살인에 기본 무기징역, 최  ...중략...
newsId: 115
newsTitle: 성범죄자 취업 최대 10년간 금지
newsDate: 2018-01-01
newsBody: 7월부터 아동·청소년 관련 52만 곳 헌재서 문제된 부분 개정해 보완 병원·유치원·학교 등  ...중략...
newsId: 116
newsTitle: [J report] “문제는 배터리가 아니야” … 아이폰이 놓친 건 혁신과 신뢰
newsDate: 2018-01-01
newsBody: 배터리 게이트로 시험대 오른 애플 성능 저하 예상되는 SW 업데이트 소비자에게 ‘진실’ 은  ...중략...
newsId: 117
newsTitle: 82년 임술년 검사 생활 시작…‘원숭이띠’ 박원순, 개띠해 인연
newsDate: 2018-01-01
newsBody: 갑술년 1994년 참여연대 창립 주도 병술년 2006년 희망제작소 창립 무술년 2018년엔  ...중략...
newsId: 118
newsTitle: “치료 도와줄게” 친구 딸 상습 성추행한 40대 항소심서 감형
newsDate: 2018-01-01
newsBody: 친구 딸을 상습 성추행한 40대 남성에게 징역 3년6월이 선고됐다. [중앙포토]
친구의 딸  ...중략...
newsId: 119
newsTitle: 마약 투여 혐의로 체포된 저가항공사 조종사
newsDate: 2018-01-01
newsBody: 2014년 5월 2일 말레이시아 쿠알라룸푸르 국제공항에 인도네시아 저가항공사인 라이온에어의  ...중략...
newsId: 120
newsTitle: 3%성장 기대 … 단기성적 취해 개혁 지연되면 안 돼
newsDate: 2018-01-01
newsBody: 국제유가 배럴당 60달러선 유력 중동 정세따라 값 요동칠 가능성

newsId: 170
newsTitle: [전문]北 김정은 2018년 신년사
newsDate: 2018-01-01
newsBody: 사랑하는 온 나라 인민들과 영명한 인민군 장병들 동포 형제 여러분!오늘 우리 모두는 근면하  ...중략...
newsId: 171
newsTitle: 1톤 트럭 방앗간 돌진…음주 측정 요구하자 주먹질
newsDate: 2018-01-01
newsBody: 주말·휴일 사건사고 잇따라
교통사고를 낸 운전자가 출동한 경찰을 때려 입건되는 등 주말과   ...중략...
newsId: 172
newsTitle: [뉴스 그 후] '진돗개 숭배·아동학대' 패륜집단 어떻게 됐나
newsDate: 2018-01-01
newsBody: ‘황금개띠의 해’로 불리는 무술년(戊戌年) 새해가 밝은 가운데 지난해 온 나라를 충격에 빠  ...중략...
newsId: 173
newsTitle: 식당서 행패 등 두번째 물의 빚은 경남도의원 약식기소
newsDate: 2018-01-01
newsBody: 만취 상태에서 손님에게 행패를 부리는 등 두 번째 물의를 빚은 경남도의원이 벌금형 처벌을   ...중략...
newsId: 174
newsTitle: "하차 요구 무시하고 계속 달린 택시, 감금죄 안 돼"
newsDate: 2018-01-01
newsBody: 내려달라는 승객의 요구를 무시하고 주행해 감금한 혐의로 재판에 넘겨진 택시기사에 대해 법원  ...중략...
newsId: 175
newsTitle: 文 대통령, 의인 6명과 북한산 등반으로 무술년 새해맞이
newsDate: 2018-01-01
newsBody: 문재인 대통령이 1일 시민과 함께 북한산을 등반하며 무술년(戊戌年) 새해 첫날을 맞이했다.  ...중략...
newsId: 176
newsTitle: 3남매 숨지게 한 친어머니 구속영장 신청
newsDate: 2018-01-01
newsBody: 12월31일 새벽 2시28분께 광주광역시 북구 두암동 한 아파트에서 불이 나 5살·3살 남  ...중략

newsId: 226
newsTitle: 안방극장 ‘이름값’ 전쟁 후끈· 전시장 대가들 회고전 봇물
newsDate: 2018-01-01
newsBody: 한국전쟁을 배경으로 스윙댄스에 빠져드는 북한군이라는 독특한 소재가 눈에 띄는 2018년 신  ...중략...
newsId: 227
newsTitle: 검찰, 박근혜 '국정원 뇌물' 곧 추가 기소
newsDate: 2018-01-01
newsBody: 박근혜 전 대통령이 국가정보원 특수활동비를 일종의 뇌물로 받은 혐의로 곧 추가 기소될 것으  ...중략...
newsId: 228
newsTitle: "북한-중국`러시아 석유 밀수 실태 문서 확인"
newsDate: 2018-01-01
newsBody: 중국과 러시아 기업이 가담해 북한에 석유류 제품을 몰래 팔아온 실태가 문서로 확인됐다고 일  ...중략...
newsId: 229
newsTitle: 요미우리 "북한-중·러 석유류 밀수 네트워크 문서로 확인돼"
newsDate: 2018-01-01
newsBody: [아시아경제 전경진 기자] 중국과 러시아 기업이 북한에 석유류 제품을 몰래 팔아온 실태가   ...중략...
newsId: 230
newsTitle: 2018 첫 구속 의원, 최경환? 이우현?… 첫 주에 결정될 듯
newsDate: 2018-01-01
newsBody: 뇌물수수 혐의로 구속영장이 청구된 자유한국당 최경환 이우현 의원의 구속 여부가 새해 첫 주  ...중략...
newsId: 231
newsTitle: [2018 유통 톺아보기 ②] “아웃렛ㆍ복합쇼핑몰 너마저”…新성장동력 규제 벽에 막히나
newsDate: 2018-01-01
newsBody: -성장 멈춘 백화점…돌파구 마련에 고심
-‘체험형’ 아웃렛ㆍ복합쇼핑몰로 눈 돌린다
-내년에  ...중략...
newsId: 232
newsTitle: 광주 아파트서 화재 3남매 사망…경찰, 母 긴급체포
newsDate: 2018-01-01
newsBody: 광주의 한 아파트 화재로 어린 3남매가 숨

newsId: 282
newsTitle: [2018 서울신문 신춘문예 단편소설 당선작] 플랫폼 (김민수)
newsDate: 2018-01-01
newsBody: 사비는 순서를 기다린다. 복도의 고요함은 일부러 꾸며진 듯하다. 문이 닫히는 소리. 누군가  ...중략...
newsId: 283
newsTitle: [김욱동 창문을 열며] 붉은 감이 아니어도
newsDate: 2018-01-01
newsBody: 김욱동 문화평론가·UNIST 초빙교수
조선시대 선조 때 활약한 무인이자 시인인 노계 박인로  ...중략...
newsId: 284
newsTitle: 이영주 민노총 사무총장 구속
newsDate: 2018-01-01
newsBody: 수배 2년 만에 경찰에 체포된 민주노총 이영주(52·여) 사무총장이 구속됐다.
서울중앙지법  ...중략...
newsId: 285
newsTitle: 美中 패권경쟁·북핵 완성…불확실한 갈림길 선 2018년
newsDate: 2018-01-01
newsBody: 美 안보·중간선거 등 외교 시험대
中 시진핑 2기 ‘1인 천하’ 본격화
유럽 ‘포퓰리즘 당  ...중략...
newsId: 286
newsTitle: 화마가 삼남매 삼킬 때…만취 엄마 혼자 베란다 피신
newsDate: 2018-01-01
newsBody: ‘손·발 화상’ 친모 과실치사 체포
발화점 가스불 → 담뱃불 말 바꿔
화재 전 前남편에 자  ...중략...
newsId: 287
newsTitle: 다스 회계 실무자 소환한 檢… 비자금 조성 배경 수사 고삐
newsDate: 2018-01-01
newsBody: BBK 송금 맡았던 MB 최측근
검찰, 다스 회계 실무자 소환…MB 정조준 서울신문DB,   ...중략...
newsId: 288
newsTitle: ‘적폐수사’ 속도… 연초 6~7명 기소할 듯
newsDate: 2018-01-01
newsBody: 檢, 연말연시 연휴에도 출근 박차
‘다스 비자금 의혹’ 등 적폐청산 수사를 하고 있는 서울  ...중략...
n

In [1]:
# 기간 설정 시 한달 단위로 쪼개기
fromdate = "\'2018-01-01\'"
todate ="\'2022-11-31\'"

day31 = [1,3,5,7,8,10,12]; day30 = [2,4,6,9,11]

from_

1번째 글
